# Process wheel data into movements 

In [2]:
""" 
IMPORTS
"""
import os
import autograd.numpy as np
import pickle
from collections import defaultdict
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from scipy import stats
from datetime import datetime
from scipy.interpolate import interp1d
from numba import njit, prange 
from one.api import ONE
import concurrent.futures

# Get my functions
functions_path =  '/home/ines/repositories/representation_learning_variability/Functions/'
functions_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/'
os.chdir(functions_path)
from data_processing import process_quiescence
from wheel_functions import wheel_trial_epoch
one = ONE(base_url='https://alyx.internationalbrainlab.org')

## Load training data

In [23]:
data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/'
data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/'
os.chdir(data_path)
session_all = pickle.load(open('training_2024-04-23', 'rb'))

In [3]:
# Choose a session with good QC
data_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/'
data_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Video and wheel/'
# save_path = '/home/ines/repositories/representation_learning_variability/Exported figures/'

os.chdir(data_path)
pass_qc = pickle.load(open(data_path + "good_brainwide_sessions_metadata", "rb"))
pass_qc = pickle.load(open(data_path + "good_dlc_bwm_sessions_metadata", "rb"))
mouse_names = pass_qc['subject_nickname'].unique()

In [25]:
# def process_wheel_movements(mouse_data):

#     training_session = 1
#     threshold = 0.2 # Need to check if this makes sense
#     min_period = 400 # This is to match the minimum length of quiescence periods
#     bin_size = 0.05
#     if training_session == 'last':
#         mouse_session_num = np.max(mouse_data['training_day'])
#         use_data = mouse_data.loc[mouse_data['training_day'] >= mouse_session_num]
#     else:
#         use_data = mouse_data.loc[mouse_data['training_day'] == training_session]

#     # Loop through sessions
#     mouse_sessions = use_data['session'].unique()
#     for s, session in enumerate(mouse_sessions):
        
#         session_trials = use_data.loc[use_data['session']==session]
#         session_trials['index'] = np.arange(0, len(session_trials))
        
#         df = wheel_trial_epoch(one, session_trials, session, bin_size, threshold, min_period).dropna()  

#         # Save to a unique file
#         pickle.dump(df, open("wheel_movements_" + str(training_session) + '_' + session, "wb"))


# def process_quiescence_parallel(mouse_data):

#     training_session = 1

#     if training_session == 'last':
#         mouse_session_num = np.max(mouse_data['training_day'])
#         use_data = mouse_data.loc[mouse_data['training_day'] >= mouse_session_num]
#     else:
#         use_data = mouse_data.loc[mouse_data['training_day'] == training_session]
        
#     # Loop through sessions
#     mouse_sessions = use_data['session'].unique()
#     for s, session in enumerate(mouse_sessions):
        
#         wheel_data = pickle.load(open('wheel_movements_' + str(training_session) + '_' + session, "rb"))
#         processed_quiescence = process_quiescence(wheel_data)

#         # Save to a unique file
#         pickle.dump(processed_quiescence, open("processed_quiescence_" + str(training_session) + '_' + session, "wb"))


# def parallel_process_data(session_all, mouse_names, function_name):
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         # Split data into chunks if needed
#         chunks = [session_all[i] for i in mouse_names]  # Split data into chunks

#         # Process each chunk in parallel
#         executor.map(function_name, chunks)


In [4]:
def process_wheel_movements(mouse_name):

    data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Trial_data/'
    os.chdir(data_path)

    training_session = 'last'
    threshold = 0.2 # Need to check if this makes sense
    min_period = 400 # This is to match the minimum length of quiescence periods
    bin_size = 0.05
    mouse_data = pickle.load(open('training_data_trials_'+ mouse_name, "rb"))

    if training_session == 'last':
        mouse_session_num = np.max(mouse_data['training_day'])
        use_data = mouse_data.loc[mouse_data['training_day'] >= mouse_session_num]
    else:
        use_data = mouse_data.loc[mouse_data['training_day'] == training_session]

    data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Wheel/'
    os.chdir(data_path)

    # Loop through sessions
    mouse_sessions = use_data['session'].unique()
    for s, session in enumerate(mouse_sessions):
        
        session_trials = use_data.loc[use_data['session']==session]
        session_trials['index'] = np.arange(0, len(session_trials))
        
        df = wheel_trial_epoch(one, session_trials, session, bin_size, threshold, min_period).dropna()  

        # Save to a unique file
        pickle.dump(df, open("wheel_movements_" + str(training_session) + '_' + session, "wb"))


def process_quiescence_parallel(mouse_name):

    data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Trial_data/'
    os.chdir(data_path)

    training_session = 'last'
    mouse_data = pickle.load(open('training_data_trials_'+ mouse_name, "rb"))

    if training_session == 'last':
        mouse_session_num = np.max(mouse_data['training_day'])
        use_data = mouse_data.loc[mouse_data['training_day'] >= mouse_session_num]
    else:
        use_data = mouse_data.loc[mouse_data['training_day'] == training_session]

    # Loop through sessions
    mouse_sessions = use_data['session'].unique()
    for s, session in enumerate(mouse_sessions):
        data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Wheel/'
        os.chdir(data_path)
        wheel_data = pickle.load(open('wheel_movements_' + str(training_session) + '_' + session, "rb"))
        processed_quiescence = process_quiescence(wheel_data)

        # Save to a unique file
        data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Wheel/processed/'
        os.chdir(data_path)
        pickle.dump(processed_quiescence, open("processed_quiescence_" + str(training_session) + '_' + session, "wb"))


def parallel_process_data(mouse_names, function_name):
    with concurrent.futures.ThreadPoolExecutor() as executor:

        # Process each chunk in parallel
        executor.map(function_name, mouse_names)


In [6]:
# Loop through animals
function_name = process_wheel_movements
function_name = process_quiescence_parallel
# data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Wheel/'
# os.chdir(data_path)
parallel_process_data(mouse_names, function_name)

47
111
132
189
257
269
319
275
210
392
274
322
83
360
286
404
370
238
244
253
500
494
495
296
339
373
368
399
508
427
429
448
470
485
513
514
533
534
17
35
561
569
576
578
509
593
594
89
220
103
245
687
565
153
63
79
606
357
459
262
265
732
734
258
0
1
319
20
335
28
31
289
34
47
52
59
60
370
66
68
377
379
380
382
383
428
87
393
93
398
442
333
102
112
126
364
149
152
178
250
390
186
188
208
212
220
222
223
224
226
230
234
424
236
243
254
277
279
462
302
303
312
313
314
321
325
343
344
516
372
353
373
374
392
395
398
402
409
410
414
415
420
436
458
459
461
463
466
467
472
482
492
495
508
521
522
492
523
544
546
547
548
551
554
555
562
573
527
577
710589

595
599
613
617
620
625
628
634
637
640
641
642
644
645
646
650
653
654
635
93
94
95
96
377
97
1
2
3
4
9
10
14
17
20
21
22
23
25
29
31
34
39
43
44
46
50
52
56
60
62
65
67
71
74
78
80
87
88
91
100
103
104
106
107
114
115
119
122
123
135
138
144
147
149
154
159
160
168
173
177
185
188
195
198
202
206
208
210
213
215
218
221
229
233
234
235